### Setup

Read the data into a dataframe

In [15]:
import pandas as pd
import csv

data = {"source":[], "dest_ip":[], "dest_owner":[], "type":[], "OS": [], "length":[], "app":[], "category":[]}
with open("parsed_2.csv", "r") as inf:
    reader = csv.reader(inf, delimiter=',', quotechar = '"')
    next(reader)
    for row in reader:
        data["source"].append(row[0])
        data["dest_ip"].append(row[1])
        owner = row[2][0:row[2].find(" ")].strip()
        if owner.find("AKAMAI") != -1:
            owner = "AKAMAI"
        data["dest_owner"].append(owner)
        data["type"].append(row[3])
        data["OS"].append(row[4])
        data["length"].append(row[5])
        data["app"].append(row[6])
        data["category"].append(row[7])
df = pd.DataFrame(data)
print(df.head(15))


         source         dest_ip                dest_owner type   OS length  \
0   192.168.2.4   17.120.254.12        APPLE-ENGINEERING,  tcp  ios     71   
1   192.168.2.4   17.120.254.12        APPLE-ENGINEERING,  tcp  ios     40   
2   192.168.2.4   17.120.254.12        APPLE-ENGINEERING,  tcp  ios     40   
3   192.168.2.4  17.248.128.144        APPLE-ENGINEERING,  tcp  ios     83   
4   192.168.2.4  17.248.128.143        APPLE-ENGINEERING,  tcp  ios     83   
5   192.168.2.4  17.248.128.173        APPLE-ENGINEERING,  tcp  ios     83   
6   192.168.2.4  17.248.128.144        APPLE-ENGINEERING,  tcp  ios     52   
7   192.168.2.4  17.248.128.143        APPLE-ENGINEERING,  tcp  ios     52   
8   192.168.2.4  17.248.128.173        APPLE-ENGINEERING,  tcp  ios     52   
9   192.168.2.4  17.248.128.144        APPLE-ENGINEERING,  tcp  ios     52   
10  192.168.2.4  17.248.128.173        APPLE-ENGINEERING,  tcp  ios     52   
11  192.168.2.4  17.248.128.143        APPLE-ENGINEERING,  tcp  

### Top 15 most frequent providers (based on packet count)

In [16]:
top_fifteen = df[df.dest_owner != ""]["dest_owner"].value_counts()[0:15]
print(top_fifteen)

AMAZON-02,                      64856
AKAMAI                          57680
GOOGLE,                         22274
HIGHWINDS3,                      9759
CMCS,                            8538
CLOUDFLARENET,                   6397
AMAZON-AES,                      5911
EDGECAST,                        4628
FASTLY,                          4185
FACEBOOK,                        3933
LEVEL3,                          3737
APPLE-ENGINEERING,               2113
MOPUB,                            916
MICROSOFT-CORP-MSN-AS-BLOCK,      908
PAYPAL,                           868
Name: dest_owner, dtype: int64


### Scores

Previous analysis does not take into account the differences in number of packets for each app (this is why roblox is so high). 
Let's rank providers based on the proportion of packets captured for each app. 


Score for a given provider is given by  $$Score = \frac{1}{N} \sum_{n=1}^{N} \frac{P_n}{T_n}$$

Where $N$ is the number of apps, $P_n$ is the number of outgoing packets for app $n$ handled by the provider, and $T_n$ is the total number of outgoing packets for app $n$. Thus, the maximum possible score (all packets handled for all apps) is 1.

In [17]:
def get_scores(df):
    owner_dict = {key:0 for key in df.dest_owner.unique()}
    df = df[df.dest_owner != ""]
    
    
    for app in df.app.unique():
        perc = df[df.app == app]["dest_owner"].value_counts()
        total = len(df[df.app == app])
        perc = {key: float(value/total) for key, value in perc.items()}
        for key,value in perc.items(): owner_dict[key] += value
        
    apps = len(df.app.unique())
    score = {key: float(value)/apps for key,value in owner_dict.items()}
    scores = pd.DataFrame(sorted(score.items(), key=lambda x: x[1], reverse=True), columns = ["Provider", "Score"])
    scores["% Traffic"] = scores["Provider"].map(lambda x: len(df[df.dest_owner == x])/len(df)*100)
    return scores

print(get_scores(df).head(15))

              Provider     Score  % Traffic
0           AMAZON-02,  0.228789  32.430594
1              GOOGLE,  0.183949  11.137891
2               AKAMAI  0.138793  28.842307
3            FACEBOOK,  0.072727   1.966657
4          AMAZON-AES,  0.068440   2.955736
5   APPLE-ENGINEERING,  0.063912   1.056585
6                CMCS,  0.047298   4.269342
7       CLOUDFLARENET,  0.041984   3.198756
8          HIGHWINDS3,  0.040244   4.879890
9              FASTLY,  0.035119   2.092667
10           EDGECAST,  0.018572   2.314185
11            BADOO-U,  0.011181   0.133011
12             PAYPAL,  0.010092   0.434035
13             LEVEL3,  0.008187   1.868649
14             SQUARE,  0.006264   0.194516


### Shannon's Entropy

Now let's analyze the distribution of packet providers. Let's calculate Shannon's entropy for each app. This value will be highest if providers are evenly distributed. 

In [18]:
from scipy.stats import entropy

app_dict = {app:None for app in df.app.unique()}
for app in df.app.unique():
    counts = [len(df[df["app"] == app][df["dest_owner"] == dest_owner]) for dest_owner in df.dest_owner.unique()]
    app_dict[app] = entropy(counts, base=None)
app_entropy = pd.DataFrame(sorted(app_dict.items(), key=lambda x: x[1], reverse=True), columns = ['App', 'Entropy'])
app_entropy["CDN Count"] = [len(df[df.app == app].dest_owner.unique()) for app in app_entropy['App']]
app_entropy["Top CDN"] = [df[df.app == app]['dest_owner'].value_counts().idxmax() for app in app_entropy['App']]
print(app_entropy)

/Users/lavanyasingh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


               App   Entropy  CDN Count             Top CDN
0        woodshop2  2.105207         17         HIGHWINDS3,
1   watermarbling2  2.093986         16         HIGHWINDS3,
2        textfree2  2.068500         14          AMAZON-02,
3        epicrace2  2.052115         17          AMAZON-02,
4     woodturning2  1.870441         19           EDGECAST,
..             ...       ...        ...                 ...
56     googlehome2  0.258126          3             GOOGLE,
57      lingokids2  0.104512          8          AMAZON-02,
58         noggin2  0.099769         10              AKAMAI
59    youtubekids2  0.067905          3             GOOGLE,
60       baseline2  0.000000          1  APPLE-ENGINEERING,

[61 rows x 4 columns]


**15 apps with the lowest entropy**

In [20]:
print(app_entropy.sort_values(by=['Entropy']).head(15))

              App   Entropy  CDN Count             Top CDN
60      baseline2  0.000000          1  APPLE-ENGINEERING,
59   youtubekids2  0.067905          3             GOOGLE,
58        noggin2  0.099769         10              AKAMAI
57     lingokids2  0.104512          8          AMAZON-02,
56    googlehome2  0.258126          3             GOOGLE,
55  pbskidsvideo2  0.263238          6          AMAZON-02,
54     googleduo2  0.288033          4             GOOGLE,
53          epic2  0.535533          6         HIGHWINDS3,
52      abcmouse2  0.542091         12          AMAZON-02,
51      whatsapp2  0.549062          4           FACEBOOK,
50     messenger2  0.574133          7           FACEBOOK,
49         hinge2  0.578442          8              AKAMAI
48        tiktok2  0.632064          9              AKAMAI
47      poshmark2  0.640763         10          AMAZON-02,
46       youtube2  0.706418          5             GOOGLE,


**15 app with the highest entropy**

In [21]:
print(app_entropy.sort_values(by=['Entropy'], ascending=False).head(15))

               App   Entropy  CDN Count      Top CDN
0        woodshop2  2.105207         17  HIGHWINDS3,
1   watermarbling2  2.093986         16  HIGHWINDS3,
2        textfree2  2.068500         14   AMAZON-02,
3        epicrace2  2.052115         17   AMAZON-02,
4     woodturning2  1.870441         19    EDGECAST,
5     creditkarma2  1.831104          8      GOOGLE,
6       braintest2  1.688237         16      LEVEL3,
7        idplease2  1.688140         17  HIGHWINDS3,
8            ebay2  1.657671          9       AKAMAI
9       foxsports2  1.638762         16       AKAMAI
10       turbotax2  1.589736         11   AMAZON-02,
11           wish2  1.583354         10   AMAZON-02,
12       kiddopia2  1.580716          8    FACEBOOK,
13        life3602  1.576536         10   AMAZON-02,
14         costar2  1.528751          7  AMAZON-AES,


**Now let's get entropy by category.**

In [22]:
# making provider names shorter
df["dest_owner"] = [provider[0:15] for provider in df.dest_owner]

cat_dict = {category:None for category in df["category"].unique()}
for category in df.category.unique():
    counts = [len(df[df["category"] == category][df["dest_owner"] == dest_owner]) for dest_owner in df.dest_owner.unique()]
    counts = list(filter(lambda x: x!= 0, counts))
    cat_dict[category] = entropy(counts, base=None)
cat_entropy = pd.DataFrame(cat_dict.items(), columns = ['Category', 'Entropy'])
cat_entropy.sort_values(by=["Entropy"])
cat_entropy["CDN Count"] = [len(df[df.category == category].dest_owner.unique()) for category in cat_entropy['Category']]
cat_entropy["Top CDN"] = [df[df.category == category]['dest_owner'].value_counts().idxmax() for category in cat_entropy['Category']]
cat_entropy["# of Apps"] = [len(df[df.category == category].app.unique()) for category in cat_entropy['Category']]
print(cat_entropy)
    

/Users/lavanyasingh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


           Category   Entropy  CDN Count          Top CDN  # of Apps
0               all  2.204560         32           AKAMAI         10
1              kids  1.162608         19       AMAZON-02,         10
2          shopping  2.056382         23       AMAZON-02,         10
3           finance  2.111467         20       AMAZON-02,         10
4                na  0.000000          1  APPLE-ENGINEERI          1
5  socialnetworking  2.196962         16          GOOGLE,         10
6         lifestyle  1.905735         19       AMAZON-02,         10


Keep in mind that the sample size for each category is still pretty tiny.

### Relationship between entropy and CDN

Let's see if there's a correlation between entropy and number of CDN's used. 

In [29]:
app_entropy.corr()

,Entropy,CDN Count
Entropy,1.000000,0.614213
CDN Count,0.614213,1.000000


Let's calculate the average entropy of the apps that each CDN dominates (is the most used CDN for).

In [38]:
provider_df = pd.DataFrame(df["dest_owner"].unique(), columns=['provider'])
entropy = []
for provider in provider_df["provider"]:
    try:
        entropy.append(sum(app_entropy[app_entropy["Top CDN"]==provider]["Entropy"])/len(app_entropy[app_entropy["Top CDN"]==provider]))
    except ZeroDivisionError:
        entropy.append(-1)
provider_df["entropy"] = entropy
print(provider_df[provider_df["entropy"] != -1])

          provider   entropy
2        FACEBOOK,  1.021737
3            CMCS,  1.153058
4          GOOGLE,  0.922011
5      AMAZON-AES,  1.402623
8           AKAMAI  1.137157
10      AMAZON-02,  1.224780
14     HIGHWINDS3,  1.605717
16  CLOUDFLARENET,  1.339625
18       EDGECAST,  1.669380
19         FASTLY,  1.296878
26         PAYPAL,  1.466309
36        BADOO-U,  1.154267
37         SQUARE,  1.457684
52         LEVEL3,  1.688237


Small providers appear to dominate for apps with high entropy (even distribution). Need to more thoroughly test this hypothesis.

to do: ANOVA to see if mean entropy per provider is independent? or maybe mean entropy per category?

### Conclusion

Conclusions: There's a good amount of variance in the entropy of each category. Kids and games have the lowest entropy and use akamai the most - would be interesting to examine correlation between entropy and top CDN and between category and top CDN. I think for future data collection we should also collect information about the size of the company that owns the app, the company itself (to check 3p versus 1p CDN), and the country of the app's owner (I didn't analyze that here, but TikTok talks to a Chinese CDN whereas the rest talk to American ones and that might be interesting).


How many CDN's do apps tend to use? The more CDN's, the more chances for failure.

In [24]:
counts = {}
for app in df.app.unique():
    counts.update({app:len(df[df["app"] == app].dest_owner.unique())})
counts_df = pd.DataFrame(counts.items(), columns=["app", "# of CDN's"])    
counts_df = counts_df.sort_values(by="# of CDN's", ascending=False)
counts_df["category"] = [df[df['app'] == app].category.unique()[0] for app in counts_df['app']]
print(counts_df.head(15))

               app  # of CDN's          category
43    woodturning2          19               all
0        woodshop2          17               all
49       idplease2          17               all
1        epicrace2          17               all
58      braintest2          16               all
51  watermarbling2          16               all
8       foxsports2          16               all
4      parkmaster2          14               all
45       textfree2          14         lifestyle
18           nike2          13          shopping
6           venmo2          12           finance
46       abcmouse2          12              kids
40           yubo2          11  socialnetworking
31       turbotax2          11           finance
27         nebula2          11         lifestyle
